In [1]:
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

In [2]:
!pip install textsearch
!pip install contractions
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sameernawaz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sameernawaz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
categories = ['airlines', 'celebrities', 'colleges', 'fastfood', 
              'leagues', 'news', 'streamingplatforms', 'techgiants']

subCategories = [['output_AmericanAir', 'output_JetBlue', 'output_qatarairways', 'output_SingaporeAir'], 
                 ['output_TheRock', 'output_ArianaGrande', 'output_chrissyteigen', 'output_jimmyfallon'], 
                 ['output_Stanford', 'output_Harvard', 'output_MIT', 'output_UCBerkeley'], 
                 ['output_McDonalds', 'output_BurgerKing', 'output_Starbucks', 'output_wendys'], 
                 ['output_IPL', 'output_MLB', 'output_MLS', 'output_NBA'], 
                 ['output_BBCWorld', 'output_CNN', 'output_washingtonpost', 'output_nytimes'], 
                 ['output_Netflix', 'output_Spotify', 'output_PrimeVideo', 'output_HBO'], 
                 ['output_amazon', 'output_facebook', 'output_Tesla', 'output_Microsoft']]

In [4]:
sizeOfSets = 400

categCount = 0
for i in range(len(categories)):
    for j in subCategories[i]:
        categCount += 1
        
categCount *= sizeOfSets

val = [None] * categCount # Keep value (sizeOfSets * no. of categories)
sizeOfTweetContent = 75

print(len(val)) # No. Of Posts

12800


In [5]:
def inputDataInDataframe():
    overCount= 0
    for i in range(len(categories)):
        for j in subCategories[i]:
            direc:str = '/Users/sameernawaz/AnacondaProjects/Datasets/tweets/'+categories[i]+'/'+j+'.csv'
            with open(direc, 'r') as csv_file:
                csv_reader = csv.reader(csv_file)
                count = 0
                for line in csv_reader:
                    if len(line[6]) > sizeOfTweetContent:
                        val[overCount] = line
                        count += 1
                        overCount += 1
                        if count >= sizeOfSets :
                            break

In [6]:
inputDataInDataframe()

In [7]:
df = pd.DataFrame(val, columns = ['date', 'username', 'to', 'replies', 
                                  'retweets', 'favorites', 'text', 'geo', 
                                  'mentions', 'hashtags', 'id', 'permalink'])

In [8]:
# Once data added in dataframe, Shuffle it
df = df.sample(frac=1).reset_index(drop=True)
# Add a unique id (rec_id) for all tweets
df = df.assign(rec_id=np.arange(len(df))).reset_index(drop=True)

In [9]:
df = df[['rec_id', 'username', 'text', 'date', 'retweets']]
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12800 entries, 0 to 12799
Data columns (total 5 columns):
rec_id      12800 non-null int64
username    12800 non-null object
text        12800 non-null object
date        12800 non-null object
retweets    12800 non-null object
dtypes: int64(1), object(4)
memory usage: 600.0+ KB


In [10]:
import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['text']))
len(norm_corpus)

12800

In [11]:
# Extract TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(12800, 26984)

In [12]:
# Compute Pairwise Document Similarity

from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.iloc[11351, 12423]

0.0

In [13]:
# Get list of post IDs

posts_list = df['rec_id'].values
posts_list, posts_list.shape

(array([    0,     1,     2, ..., 12797, 12798, 12799]), (12800,))

In [14]:
posts_idx = np.where(posts_list == 11351)[0][0]
posts_idx

11351

In [15]:
posts_similarities = doc_sim_df.iloc[posts_idx].values
posts_similarities

array([0., 0., 0., ..., 0., 0., 0.])

In [16]:
# Get top 5 similar post IDs

similar_posts_idxs = np.argsort(-posts_similarities)[1:6]
similar_posts_idxs

array([11630,  3469, 12791,  5389, 11049])

In [17]:
similar_posts = posts_list[similar_posts_idxs]
similar_posts

array([11630,  3469, 12791,  5389, 11049])

In [18]:
def relevant_posts(postId, returnSize, posts = posts_list, doc_sims = doc_sim_df):
    # find post id
    posts_idx = np.where(posts == postId)[0][0]
    # get posts similarities
    posts_similarities = doc_sims.iloc[posts_idx].values
    # get top 5 similar post IDs
    similar_posts_idxs = np.argsort(-posts_similarities)[1:returnSize+1]
    # get top 5 posts
    similar_posts = posts[similar_posts_idxs]
    # return the top 5 posts
    return similar_posts

In [52]:
post_id = 11323
returnValSize = 6

print('-----Input Post-----')
print((df.loc[df['rec_id'] == post_id]['text']).values[0])
print('\n-----Relevant Posts-----')
for i in relevant_posts(postId = post_id, returnSize = returnValSize, posts = posts_list, doc_sims = doc_sim_df):
    print("=>",(df.loc[df['rec_id'] == i]['text']).values[0])

-----Input Post-----
With #MLSCup in their sights, @LAFC turn their attention to Seattle: http://soc.cr/h9bu50wUB4y #LAFC

-----Relevant Posts-----
=> #LAFC #LAGalaxy. Meant to be. @LAFC relishing the chance to take down their cross-town rivals in the Audi #MLSCupPlayoffs: http://soc.cr/LePJ50wRKGv
=> ∙ Kelyn Rowe Seattle ∙ Juan Agudelo LAFC @MattDoyle76 breaks down the moves he 𝑤𝑎𝑛𝑡𝑠 to see. http://soc.cr/Z1Mz50xksGE
=> A sellout in Seattle? You betcha! http://soc.cr/eoNe50x09zd #MLSCup // : @SoundersFC
=> Will @LAFC continue their historic season and make it to #MLSCup? #LAFCvSEA // Audi #MLSCupPlayoffs
=> They met twice in the regular season... #LAFC 4-1 #Sounders #Sounders 1-1 #LAFC Our guys break down those meetings and look ahead to tonight. #LAFCvSEA // Audi #MLSCupPlayoffs
=>  Single-season scoring record Supporters' Shield MVP #MLSCup It's been a magical season for @11carlosV and @lafc.


In [20]:
# Sort Posts

post_id = 90
returnValSize = 15

postsList = []

print('-----Input Post-----')
print((df.loc[df['rec_id'] == post_id]['text']).values[0])
print('\n-----Relevant Posts-----')
for i in relevant_posts(postId = post_id, returnSize = returnValSize, posts = posts_list, doc_sims = doc_sim_df):
    postsList.append((df.loc[df['rec_id'] == i]).values[0])

-----Input Post-----
Analysis: The latest Ukraine revelation spotlights a major gap in Trump’s defense

-----Relevant Posts-----


In [21]:
def sortDate(val): 
    return val[3]

def sortRetweet(val): 
    return val[4]

postsList.sort(key=sortDate, reverse=True)

recentList = postsList[0:5]

for i in recentList:
    print("=>", i[2])

=> Perspective: Trump’s xenophobia is an American tradition — but it doesn’t have to be
=> Analysis | Choose your own adventure: Can you keep from being impeached over Ukraine
=> Analysis: The unmitigated ludicrousness of Trump trying to distance himself from Giuliani’s work on Ukraine
=> Analysis: 3 takeaways from Mark Sandy’s and Philip Reeker’s testimony on Ukraine
=> Opinion: Pompeo just flirted with Trump’s Ukraine conspiracy theory. This isn’t normal, folks.
